# Домашнее задание 1

### Описание

В вашем распоряжении датасет с русскоязычными отзывами о мобильных телефонах с выставленным рейтингом от 1 до 5.
Ключевая задача – обучить любую модель регрессии (или классификации, если решите таким путём пойти) из пакетов scikit, XGBoost, LightGBM, CatBoost.


Необходимая метрика:

1. Со звёздочкой (дополнительный балл) – MAE <= 0.5
2. Минимальное допустимое значение – МАЕ <= 1.0

### Что необходимо сделать

1. Откройте датасет
2. Разделите на обучение и тест
3. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 
4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).
5. Вывести ТОП-50 наиболее частотных токенов:
- только для униграмм
- только для биграмм
- только для триграмм
- для всех вариантов n-грамм одновременно

Напишите, какие наблюдения и выводы есть.
6. Повторите пункт 5, только отдельно для отзывов с рейтингом «4-5», «3» и «1-2». Есть ли ключевые отличия? Есть кандидаты на попадание в список стоп-слов?

7. Составьте список своих ключевых слов, для помощи можно использовать nltk и punctuations из string. 
8. Закодируйте полученные отзывы с помощью CountVectorizer и TfIdfVectorizer (экспериментируйте с параметрами min_df  и max_df). 
9. Обучение одну или несколько моделей машинного обучения на разных представлениях данных
10. Валидируйте модель. Если модель соответствует условиям метрик, то работа завершена. В ином случае, экспериментируйте, начиная с пункта 7. 
11. По всем попыткам обучить качественную модель пишите свои выводы и замечания, почему так получилось.


## Комментарии исполнителя

Я изменил порядок 2 и 3 пункта, потому что так намного удобнее

## 0. Импорт библиотк, определение констант

In [138]:
import os
import pandas as pd
import numpy as np
import spacy
import re

from string import punctuation

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

from pymorphy3 import MorphAnalyzer

from nltk import ngrams
from nltk.corpus import stopwords

In [139]:
list(punctuation)

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [140]:
tqdm.pandas()

In [141]:
#!python -m spacy download ru_core_news_sm

## 1. Откройте датасет

In [142]:
if os.path.exists("data_lemma.csv"):
    df = pd.read_csv("data_lemma.csv", engine='python')
else:    
    df = pd.read_csv("data.csv")

In [143]:
df.shape

(459497, 3)

In [144]:
df.head(10)

,Review,Rating,lemma
0,3D Touch просто восхитительная вещь! Заряд дер...,5.0,3d touch просто восхитительный вещь! заряд дер...
1,"Отключается при температуре близкой к нулю, не...",4.0,"отключаться при температура близкий к нулю, не..."
2,"В Apple окончательно решили не заморачиваться,...",3.0,"в apple окончательно решить не заморачиваться,..."
3,Постарался наиболее ёмко и коротко описать все...,4.0,постараться наиболее ёмко и коротко описать вс...
4,Достойный телефон. Пользоваться одно удовольст...,5.0,достойный телефон. пользоваться один удовольст...
5,6s gold 64gb,5.0,6s gold 64gb
6,Мой первый айфон. Скажу честно- Эппл ранее ник...,5.0,мой первый айфон. сказать честно- эппл ранее н...
7,Мне очень понравилась эта модель! Во-первых кл...,5.0,я очень понравиться этот модель! во-первых кла...
8,Долгое время пользовалась iPhone 5s 16Gb. Он м...,5.0,долгий время пользоваться iphone 5s 16gb. он я...
9,"Раньше был Samsung Galaxy Alpha SM-G850F 32Gb,...",4.0,ранний быть samsung galaxy alpha sm-g850f 32gb...


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 459497 entries, 0 to 459496
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Review  459497 non-null  object 
 1   Rating  457833 non-null  float64
 2   lemma   457833 non-null  object 
dtypes: float64(1), object(2)
memory usage: 10.5+ MB


In [146]:
df = df.dropna().reset_index(drop=True)
df.head()

,Review,Rating,lemma
0,3D Touch просто восхитительная вещь! Заряд дер...,5.0,3d touch просто восхитительный вещь! заряд дер...
1,"Отключается при температуре близкой к нулю, не...",4.0,"отключаться при температура близкий к нулю, не..."
2,"В Apple окончательно решили не заморачиваться,...",3.0,"в apple окончательно решить не заморачиваться,..."
3,Постарался наиболее ёмко и коротко описать все...,4.0,постараться наиболее ёмко и коротко описать вс...
4,Достойный телефон. Пользоваться одно удовольст...,5.0,достойный телефон. пользоваться один удовольст...


In [147]:
df['Rating'].value_counts()

Rating
5.0    228086
4.0    106503
3.0     53055
2.0     35705
1.0     34484
Name: count, dtype: int64

In [148]:
df = df[(df['Rating'] >= 1) & (df['Rating'] <= 5)]

In [149]:
df['Rating'].value_counts(normalize=True)

Rating
5.0    0.498186
4.0    0.232624
3.0    0.115883
2.0    0.077987
1.0    0.075320
Name: proportion, dtype: float64

Я закомментил все что ниже, потому что такую обработку лучше делать после лемматизации

In [150]:
df.duplicated().sum()

136735

In [151]:
df[df.duplicated()].head(10)

,Review,Rating,lemma
343,"Покупал 6-ку в интернет магазине, перед этими ...",4.0,"покупать 6-ку в интернет магазине, перед этот ..."
354,Довольно долго сопротивлялся смартфонам. Да и ...,5.0,довольно долго сопротивляться смартфонам. да и...
358,"Всем заядлым ""яблочникам"" хочу сказать,что чув...",4.0,"весь заядлый ""яблочникам"" хотеть сказать,чтый ..."
359,"Отличный, как и все айфоны, но цены очень высока.",5.0,"отличный, как и всё айфоны, но цена очень высока."
361,"Очень понравился телефон в использовании, мне ...",5.0,"очень понравиться телефон в использовании, я 1..."
363,"Приятный телефон в использовании, Камера на 5,...",5.0,"приятный телефон в использовании, камера на 5,..."
364,В целом доволен покупкой. Если любите пофоткат...,4.0,в целое довольный покупкой. если любить пофотк...
365,"iPhone 6 практически идеален. Отличный дизайн,...",5.0,"iphone 6 практически идеален. отличный дизайн,..."
373,"Если вы сомневаетесь нужен ли вам iPhone, если...",5.0,"если вы сомневаться нужный ли вы iphone, если ..."
385,"Бесит, что защитные стекла все идут не на весь...",5.0,"бесит, что защитный стекло всё идти не на весь..."


In [152]:
df["Review"].value_counts()

Review
Отличный телефон                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

Основная причина дублей - короткие одинаковые комментарии, такие тубли можно исключить

In [153]:
# df = df.drop_duplicates()

In [154]:
df["Review"].value_counts()

Review
Отличный телефон                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [155]:
df[df["Review"] == "-"].value_counts()

Review  Rating  lemma
-       5.0     -        19
        3.0     -         8
        4.0     -         5
        2.0     -         3
        1.0     -         1
Name: count, dtype: int64

Исключим записи с одинаковым комментарием, но выоским разрбросом оценки

In [156]:
#bad_comments = df["Review"].value_counts()
#bad_comments = bad_comments[bad_comments >= 3]
#bad_comments.head(20)

#bad_comments = bad_comments[bad_comments['value'].isin((4,5))]

Уберём синтаксические знаки

In [164]:
punct = list(punctuation)
punct.remove(":")
punct.remove(")")
punct.remove("(")
punct = ''.join(punct)

In [162]:
df['Review']

0         3D Touch просто восхитительная вещь! Заряд дер...
1         Отключается при температуре близкой к нулю, не...
2         В Apple окончательно решили не заморачиваться,...
3         Постарался наиболее ёмко и коротко описать все...
4         Достойный телефон. Пользоваться одно удовольст...
                                ...                        
457828    удобный, всё работает отлично, звонит, играет,...
457829    прошло больше года, притензий нет, при моей на...
457830    мой первый аппарат на андроиде. На данный моме...
457831    Разбил iphone и не было желания покупать новый...
457832             Очень доволен покупкой и всем советую...
Name: Review, Length: 457833, dtype: object

In [179]:
df['Review'].str.replace(r'[' + punct + ']', '', regex=True)

0         3D Touch просто восхитительная вещь Заряд держ...
1         Отключается при температуре близкой к нулю неп...
2         В Apple окончательно решили не заморачиваться ...
3         Постарался наиболее ёмко и коротко описать все...
4          Достойный телефон Пользоваться одно удовольствие
                                ...                        
457828    удобный всё работает отлично звонит играет сни...
457829    прошло больше года притензий нет при моей нагр...
457830    мой первый аппарат на андроиде На данный момен...
457831    Разбил iphone и не было желания покупать новый...
457832                Очень доволен покупкой и всем советую
Name: Review, Length: 457833, dtype: object

## 2. Осуществите лемматизацию с помощью любого из озвученных на занятии инструментов 

### spacy

In [120]:
spacy_nlp = spacy.load("ru_core_news_sm")

In [121]:
def spacy_lemmatize_text(text):
    try:
        doc = spacy_nlp(text)
    except:
        print(f'{text}')
        return ""
    lemmatized_text = ' '.join([token.lemma_ for token in doc])
    return lemmatized_text

В этом месте у меня падала лемматизация, так я обнаружил что есть пустые комменты, но трай эксепт оставил, в пайморфек уже нет этого, потому что мусор удален

### pymorphy3

In [122]:
pymorphy3_analyzer = MorphAnalyzer()

In [123]:
def pymorphy3_lemmatize_text(text):
    lemmas = [pymorphy3_analyzer.parse(word)[0].normal_form for word in text.split()]
    return (' '.join(lemmas))

### Лемматинизируем

In [124]:
if not "lemma" in df.columns:
    print('Лемматизируем текст');
    df["lemma"] = df["Review"].progress_apply(lambda row: pymorphy3_lemmatize_text(row))
    df.to_csv("data_lemma.csv", index=False)

Лемматизируем текст


100%|██████████| 457833/457833 [33:58<00:00, 224.59it/s]


## 3 Обработка датасета после лемматизации

## 3. Разделите на обучение и тест

In [37]:
train, test = train_test_split(df, test_size=0.2, stratify=df['Rating'])
print(train.shape)
print(test.shape)

(365453, 6)
(91364, 6)


## 4. Провести эксперимент, и создать токены из униграмм, биграмм и триграмм (используйте nltk ngrams).

### Униграммы

In [42]:
unigrams = list(train['lemma'].str.split()) 

In [43]:
unigrams

[['если',
  'бы',
  'быть',
  'такой',
  'возможность,',
  'взять',
  'бы',
  'другой',
  'телефон'],
 ['проблема',
  'с',
  'русификация',
  'решиться',
  'настройка',
  '"формат',
  'региона"',
  'ура,',
  'теперь',
  'недостаток',
  'нет!!!быть',
  'тонкости..',
  'пока',
  'не',
  'понял,',
  'можно',
  'ли',
  'устанавливать',
  'приложение',
  'на',
  'флешку.',
  'с',
  'флешка',
  'видный',
  'только',
  'музыка',
  'и',
  'видео.',
  'документ',
  'офис',
  'на',
  'флешка',
  'не',
  'видит.'],
 ['телефон',
  'отличный,',
  'только',
  'качество',
  'отвратное,',
  'после',
  'айфон',
  '6',
  ',',
  'как',
  'то',
  'разочаровать',
  'именно',
  'в',
  'качество',
  'материал',
  'и',
  'сборки.',
  'сетка',
  'сверху',
  'протираться',
  ',',
  'кнопка',
  'домой',
  'жуткий',
  'пластиковый',
  'с',
  'ужасный',
  'неприятный',
  'нажатием,',
  'дисплей',
  'корявый',
  'приклеен,',
  'зазор',
  'между',
  'корпус',
  'и',
  'дисплей',
  'не',
  'равномерные,',
  'звук',
 